In [1]:
import numpy as np
import pandas as pd

import pymysql
import dotenv
import os
import pickle
import seaborn as sns
import matplotlib

from datetime import timedelta

import matplotlib.pyplot as plt

# Normalization/Standardization
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout, Conv1D, GRU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.saving import save_model

# 경고 무시 코드 추가
import warnings
warnings.filterwarnings('ignore')

def pltconfig_default() :
  sns.reset_defaults()
  %matplotlib inline

pltconfig_default()

matplotlib.rcParams

matplotlib.rcParams['font.family']

current_font_list = matplotlib.rcParams['font.family']

font_path = 'C:\\Windows\\Fonts\\batang.ttc'

kfont = matplotlib.font_manager.FontProperties(fname=font_path).get_name()

matplotlib.rcParams['font.family'] = [kfont] + current_font_list

In [2]:
St_NotEncode_data = pd.read_pickle("StandardScalar_final_data")

print(St_NotEncode_data.shape)

(17422, 9)


In [3]:
# Feature와 Label 분리하기
def Feature_Label(datafile) :
    X = datafile.iloc[:,:]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SNE_X, SNE_y = Feature_Label(St_NotEncode_data)
print(SNE_X.shape, SNE_y.shape)
SNE_X_train, SNE_X_test, SNE_y_train, SNE_y_test = train_test_split(SNE_X, SNE_y, test_size=0.2, random_state=10, shuffle=False)
print(SNE_X_train.shape, SNE_X_test.shape, SNE_y_train.shape, SNE_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 9) (17422,)
(13937, 9) (3485, 9) (13937,) (3485,)


In [4]:
# X 값 Window Dataset 구성
ds_x = tf.data.Dataset.from_tensor_slices(SNE_X_train).window(WINDOW_SIZE, stride=1, shift=1, drop_remainder=True)  # 여기서는 X값만 별도로 구성하므로 WINDOW_SIZE 값에 1을 더하지 않는다!
ds_x = ds_x.flat_map(lambda x : x.batch(WINDOW_SIZE))

In [5]:
# y 값 Window Dataset 구성
ds_y = tf.data.Dataset.from_tensor_slices(SNE_y_train[WINDOW_SIZE:])

In [6]:
train_data = tf.data.Dataset.zip((ds_x, ds_y)).batch(BATCH_SIZE)

In [7]:
for x, y in train_data.take(1) :
  print(x[:3])
  print()
  print(y[:3])

tf.Tensor(
[[[-0.11352868 -0.09386187 -0.6825146   0.09445245 -0.66581843
   -1.23511514 -0.21825161 -1.53935952  0.        ]
  [-0.11352868 -0.9225242  -1.06038321  0.4051263  -0.66581843
   -1.23511514 -0.21825161 -1.57681737  0.        ]
  [-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834  0.        ]]

 [[-0.11352868 -0.9225242  -1.06038321  0.4051263  -0.66581843
   -1.23511514 -0.21825161 -1.57681737  0.        ]
  [-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834  0.        ]
  [-0.11352868 -1.12968978 -1.06038321  0.71580015 -0.66581843
   -1.23511514 -0.21825161 -1.65922462  0.        ]]

 [[-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834  0.        ]
  [-0.11352868 -1.12968978 -1.06038321  0.71580015 -0.66581843
   -1.23511514 -0.21825161 -1.65922462  0.        ]
  [-0.11352868 -0.9225242  -1.06038321  0.80456411 -0.66581843
  

In [8]:
pd.concat([SNE_X_train, SNE_y_train], axis=1).head(10)

,0,1,2,3,4,5,6,7,8,8
0,-0.113529,-0.093862,-0.682515,0.094452,-0.665818,-1.235115,-0.218252,-1.539360,0.0,0.0
1,-0.113529,-0.922524,-1.060383,0.405126,-0.665818,-1.235115,-0.218252,-1.576817,0.0,0.0
2,-0.113529,-1.026107,-1.060383,0.538272,-0.665818,-1.235115,-0.218252,-1.629258,0.0,0.0
3,-0.113529,-1.129690,-1.060383,0.715800,-0.665818,-1.235115,-0.218252,-1.659225,0.0,0.0
4,-0.113529,-0.922524,-1.060383,0.804564,-0.665818,-1.235115,-0.218252,-1.689191,0.0,0.0
5,-0.113529,-1.026107,-1.060383,0.937710,-0.665818,-1.235115,-0.218252,-1.719157,0.0,0.0
6,-0.113529,-0.922524,-1.060383,0.893328,-0.665818,-1.235115,-0.218252,-1.734140,0.0,0.0
7,-0.113529,-0.922524,-1.060383,0.982092,0.549517,-1.235115,-0.333181,-1.741632,6000.0,6000.0
8,-0.113529,-1.129690,-1.060383,0.538272,1.764852,0.769233,-0.226741,-1.636750,16000.0,16000.0
9,-0.113529,-0.922524,-1.060383,0.005688,1.764852,-0.984572,-0.322080,-1.329596,24000.0,24000.0


In [9]:
# 위 내용을 함수 형태로 만들고 싶은 경우

def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(SNE_X_train).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(SNE_y_train[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [10]:
train_data = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SNE_X_test, SNE_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [11]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 9), (28,)


In [12]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 9]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [13]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 9]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [14]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 9]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [15]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [16]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('./Control_Weights/all_group/new_data/new_multi_stne_lstm_weight_include.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('./Control_Weights/all_group/new_data/new_multi_stne_lstm_weight_ns_include.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('./Control_Weights/all_group/new_data/new_multi_stne_rnn_weight_include.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('./Control_Weights/all_group/new_data/new_multi_stne_rnn_weight_ns_include.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('./Control_Weights/all_group/new_data/new_multi_stne_gru_weight_include.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('./Control_Weights/all_group/new_data/new_multi_stne_gru_weight_ns_include.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [17]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500
    487/Unknown - 7s 4ms/step - loss: 7614.3228 - mse: 183581680.0000 - mae: 7614.6343
Epoch 1: val_loss improved from inf to 7578.89648, saving model to ./Control_Weights/all_group/new_data\new_multi_stne_lstm_weight_include.h5
498/498 [==============================] - 10s 11ms/step - loss: 7608.2593 - mse: 183229872.0000 - mae: 7608.5713 - val_loss: 7578.8965 - val_mse: 182223824.0000 - val_mae: 7579.1904 - lr: 0.0010
Epoch 2/500
497/498 [============================>.] - ETA: 0s - loss: 7532.1855 - mse: 180479152.0000 - mae: 7532.4897
Epoch 2: val_loss improved from 7578.89648 to 7482.21289, saving model to ./Control_Weights/all_group/new_data\new_multi_stne_lstm_weight_include.h5
498/498 [==============================] - 3s 6ms/step - loss: 7534.2559 - mse: 180557952.0000 - mae: 7534.5596 - val_loss: 7482.2129 - val_mse: 178629216.0000 - val_mae: 7482.5430 - lr: 0.0010
Epoch 3/500
497/498 [============================>.] - ETA: 0s - loss: 7401.8086 - mse: 175943264.00

In [18]:
lstm_save_path = f"./Control_Models/all_group/new_data/lstm_model"

save_model(lstm_model, lstm_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/all_group/new_data/lstm_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/all_group/new_data/lstm_model\assets


In [19]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
    496/Unknown - 4s 2ms/step - loss: 7609.7842 - mse: 183311632.0000 - mae: 7610.1089
Epoch 1: val_loss improved from inf to 7587.61084, saving model to ./Control_Weights/all_group/new_data\new_multi_stne_rnn_weight_include.h5
498/498 [==============================] - 5s 5ms/step - loss: 7613.2871 - mse: 183415760.0000 - mae: 7613.6123 - val_loss: 7587.6108 - val_mse: 182499856.0000 - val_mae: 7587.9312 - lr: 0.0010
Epoch 2/500
484/498 [============================>.] - ETA: 0s - loss: 7520.8325 - mse: 180699584.0000 - mae: 7521.1338
Epoch 2: val_loss improved from 7587.61084 to 7487.66992, saving model to ./Control_Weights/all_group/new_data\new_multi_stne_rnn_weight_include.h5
498/498 [==============================] - 2s 4ms/step - loss: 7542.4004 - mse: 180881296.0000 - mae: 7542.7046 - val_loss: 7487.6699 - val_mse: 178966656.0000 - val_mae: 7488.0024 - lr: 0.0010
Epoch 3/500
494/498 [============================>.] - ETA: 0s - loss: 7431.2607 - mse: 176723232.0000 -

In [20]:
rnn_save_path = f"./Control_Models/all_group/new_data/rnn_model"

save_model(rnn_model, rnn_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/all_group/new_data/rnn_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/all_group/new_data/rnn_model\assets


In [21]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
    497/Unknown - 8s 5ms/step - loss: 7615.8604 - mse: 183519568.0000 - mae: 7616.1738
Epoch 1: val_loss improved from inf to 7595.12891, saving model to ./Control_Weights/all_group/new_data\new_multi_stne_gru_weight_include.h5
498/498 [==============================] - 10s 9ms/step - loss: 7615.2432 - mse: 183488112.0000 - mae: 7615.5566 - val_loss: 7595.1289 - val_mse: 182782960.0000 - val_mae: 7595.4541 - lr: 0.0010
Epoch 2/500
492/498 [============================>.] - ETA: 0s - loss: 7554.0225 - mse: 181421568.0000 - mae: 7554.3320
Epoch 2: val_loss improved from 7595.12891 to 7520.98291, saving model to ./Control_Weights/all_group/new_data\new_multi_stne_gru_weight_include.h5
498/498 [==============================] - 3s 7ms/step - loss: 7562.4165 - mse: 181573248.0000 - mae: 7562.7271 - val_loss: 7520.9829 - val_mse: 180140480.0000 - val_mae: 7521.3042 - lr: 0.0010
Epoch 3/500
490/498 [============================>.] - ETA: 0s - loss: 7466.4092 - mse: 178382464.0000 

In [22]:
gru_save_path = f"./Control_Models/all_group/new_data/gru_model"

save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/all_group/new_data/gru_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/all_group/new_data/gru_model\assets


In [23]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 2608.140625
lstm_history Validation MSE: 25408044.0
lstm_history Validation MAE: 2608.4853515625
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 2060.443603515625
lstm_history_ns Validation MSE: 19229964.0
lstm_history_ns Validation MAE: 2060.7666015625
-------------rnn_history-------------
rnn_history Validation Loss: 1818.6595458984375
rnn_history Validation MSE: 14141369.0
rnn_history Validation MAE: 1818.9664306640625
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 1809.627197265625
rnn_history_ns Validation MSE: 14028311.0
rnn_history_ns Validation MAE: 1809.9320068359375
-------------gru_history-------------
gru_history Validation Loss: 2083.66015625
gru_history Validation MSE: 19120020.0
gru_history Validation MAE: 2083.974365234375
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 1980.1707763671875
gru_history_ns Validation MSE: 178552

In [24]:
# lstm_save_path = f"./TEST_Models/lstm_model"
# rnn_save_path = f"./TEST_Models/rnn_model"
# gru_save_path = f"./TEST_Models/gru_model"

# save_model(lstm_model, lstm_save_path, overwrite=True)
# save_model(rnn_model, rnn_save_path, overwrite=True)
# save_model(gru_model, gru_save_path, overwrite=True)

In [25]:
St_NotEncode_Basic_data = pd.read_pickle("Basic_StandardScalar_final_data")
  
print(St_NotEncode_Basic_data.shape)

(52578, 9)


In [26]:
# Feature와 Label 분리하기
def Feature_Label(datafile) :
    X = datafile.iloc[:,:-1]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SB_X, SB_y = Feature_Label(St_NotEncode_Basic_data)
print(SNE_X.shape, SNE_y.shape)
SB_X_train, SB_X_test, SB_y_train, SB_y_test = train_test_split(SB_X, SB_y, test_size=0.2, random_state=10, shuffle=False)
print(SB_X_train.shape, SB_X_test.shape, SB_y_train.shape, SB_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 9) (17422,)
(42062, 8) (10516, 8) (42062,) (10516,)


In [27]:
def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [28]:
train_data = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SB_X_test, SB_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [29]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 8), (28,)


In [30]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [31]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [32]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [33]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [34]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('./Control_Weights/all_group/base_data/base_multi_stne_lstm_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('./Control_Weights/all_group/base_data/base_multi_stne_lstm_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('./Control_Weights/all_group/base_data/base_multi_stne_rnn_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('./Control_Weights/all_group/base_data/base_multi_stne_rnn_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('./Control_Weights/all_group/base_data/base_multi_stne_gru_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('./Control_Weights/all_group/base_data/base_multi_stne_gru_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [35]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500
   1498/Unknown - 12s 5ms/step - loss: 2.0727 - mse: 23.3620 - mae: 2.4388
Epoch 1: val_loss improved from inf to 2.06376, saving model to ./Control_Weights/all_group/base_data\base_multi_stne_lstm_weight.h5
1503/1503 [==============================] - 14s 6ms/step - loss: 2.0764 - mse: 23.4412 - mae: 2.4426 - val_loss: 2.0638 - val_mse: 19.5572 - val_mae: 2.4699 - lr: 0.0010
Epoch 2/500
1495/1503 [============================>.] - ETA: 0s - loss: 1.3275 - mse: 9.5812 - mae: 1.6756
Epoch 2: val_loss did not improve from 2.06376
1503/1503 [==============================] - 8s 5ms/step - loss: 1.3319 - mse: 9.6414 - mae: 1.6802 - val_loss: 2.1633 - val_mse: 22.1685 - val_mae: 2.5378 - lr: 0.0010
Epoch 3/500
1500/1503 [============================>.] - ETA: 0s - loss: 1.2947 - mse: 9.2767 - mae: 1.6400
Epoch 3: val_loss did not improve from 2.06376
1503/1503 [==============================] - 8s 5ms/step - loss: 1.2951 - mse: 9.2791 - mae: 1.6404 - val_loss: 2.0977 - val_mse: 

In [36]:
lstm_save_path = f"./Control_Models/all_group/base_data/lstm_model"

save_model(lstm_model, lstm_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/all_group/base_data/lstm_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/all_group/base_data/lstm_model\assets


In [37]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
   1486/Unknown - 5s 2ms/step - loss: 2.1867 - mse: 25.4718 - mae: 2.5490
Epoch 1: val_loss improved from inf to 2.39239, saving model to ./Control_Weights/all_group/base_data\base_multi_stne_rnn_weight.h5
1503/1503 [==============================] - 6s 3ms/step - loss: 2.1839 - mse: 25.4154 - mae: 2.5463 - val_loss: 2.3924 - val_mse: 25.6732 - val_mae: 2.7902 - lr: 0.0010
Epoch 2/500
1478/1503 [============================>.] - ETA: 0s - loss: 1.3496 - mse: 9.7603 - mae: 1.6944
Epoch 2: val_loss did not improve from 2.39239
1503/1503 [==============================] - 4s 2ms/step - loss: 1.3609 - mse: 9.9372 - mae: 1.7062 - val_loss: 2.3971 - val_mse: 27.1181 - val_mae: 2.7890 - lr: 0.0010
Epoch 3/500
1490/1503 [============================>.] - ETA: 0s - loss: 1.3171 - mse: 9.4744 - mae: 1.6614
Epoch 3: val_loss improved from 2.39239 to 2.28384, saving model to ./Control_Weights/all_group/base_data\base_multi_stne_rnn_weight.h5
1503/1503 [==============================] -

In [38]:
rnn_save_path = f"./Control_Models/all_group/base_data/rnn_model"

save_model(rnn_model, rnn_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/all_group/base_data/rnn_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/all_group/base_data/rnn_model\assets


In [39]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
   1501/Unknown - 12s 5ms/step - loss: 2.0101 - mse: 21.7880 - mae: 2.3721
Epoch 1: val_loss improved from inf to 2.26374, saving model to ./Control_Weights/all_group/base_data\base_multi_stne_gru_weight.h5
1503/1503 [==============================] - 14s 6ms/step - loss: 2.0101 - mse: 21.7893 - mae: 2.3721 - val_loss: 2.2637 - val_mse: 21.9315 - val_mae: 2.6653 - lr: 0.0010
Epoch 2/500
1495/1503 [============================>.] - ETA: 0s - loss: 1.3173 - mse: 9.4826 - mae: 1.6632
Epoch 2: val_loss improved from 2.26374 to 2.22002, saving model to ./Control_Weights/all_group/base_data\base_multi_stne_gru_weight.h5
1503/1503 [==============================] - 9s 6ms/step - loss: 1.3199 - mse: 9.5277 - mae: 1.6659 - val_loss: 2.2200 - val_mse: 23.0397 - val_mae: 2.6167 - lr: 0.0010
Epoch 3/500
1495/1503 [============================>.] - ETA: 0s - loss: 1.2505 - mse: 8.8838 - mae: 1.5881
Epoch 3: val_loss improved from 2.22002 to 1.91409, saving model to ./Control_Weights/all

In [40]:
gru_save_path = f"./Control_Models/all_group/base_data/gru_model"

save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Control_Models/all_group/base_data/gru_model\assets


INFO:tensorflow:Assets written to: ./Control_Models/all_group/base_data/gru_model\assets


In [41]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 1.806140661239624
lstm_history Validation MSE: 17.242719650268555
lstm_history Validation MAE: 2.173893451690674
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 1.826497197151184
lstm_history_ns Validation MSE: 16.83551025390625
lstm_history_ns Validation MAE: 2.193236827850342
-------------rnn_history-------------
rnn_history Validation Loss: 1.785382866859436
rnn_history Validation MSE: 16.666061401367188
rnn_history Validation MAE: 2.1604976654052734
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 1.8793364763259888
rnn_history_ns Validation MSE: 17.67788314819336
rnn_history_ns Validation MAE: 2.241887092590332
-------------gru_history-------------
gru_history Validation Loss: 1.6748753786087036
gru_history Validation MSE: 15.489799499511719
gru_history Validation MAE: 2.054129123687744
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 1.85

In [42]:
lstm_save_path = f"./TEST_Models/base_lstm_model"
rnn_save_path = f"./TEST_Models/base_rnn_model"
gru_save_path = f"./TEST_Models/base_gru_model"

save_model(lstm_model, lstm_save_path, overwrite=True)
save_model(rnn_model, rnn_save_path, overwrite=True)
save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./TEST_Models/base_lstm_model\assets


INFO:tensorflow:Assets written to: ./TEST_Models/base_lstm_model\assets


INFO:tensorflow:Assets written to: ./TEST_Models/base_rnn_model\assets


INFO:tensorflow:Assets written to: ./TEST_Models/base_rnn_model\assets


INFO:tensorflow:Assets written to: ./TEST_Models/base_gru_model\assets


INFO:tensorflow:Assets written to: ./TEST_Models/base_gru_model\assets
